# Dimensional Analysis

This notebook prototypes a small controlled natural language for specifying physical properties.
Our example sentence is

    the ball has a mass of 5 kg and a kinetic energy of 12 mN

and we want to obtain the logical expression

    (mass theball (quant 5 kilo gram))∧(ekin theball (quant 12 meter Newton)).

However, we have to use [dimensional analysis](https://en.wikipedia.org/wiki/Dimensional_analysis)
to disambiguate whether *12 mN* stands for *12 milli Newton* or for *12 meter Newton*.
Since *milli Newton* is not an energy, we can discard it.

This example was created for a [paper submitted to CICM 2020](https://kwarc.info/people/mkohlhase/submit/cicm20-glif.pdf).

In [1]:
subdir dim_analysis

Changed to subdirectory dim_analysis

Successfully reloaded GF

### Grammar

In [2]:
abstract Grammar = {
    cat
        Object;
        Measurable;   -- kinetic energy
        Measurement;  -- a kinetic energy of 3 m N
        Measurements; -- a kinetic energy of 3 m N and a mass of 3 k g
        Unit;         -- m N
        S;            -- sentence
        
    fun
        state : Object -> Measurements -> S;
        unitCombine : Unit -> Unit -> Unit;
        
        measure : Measurable -> Int -> Unit -> Measurement;
        toMeasurements : Measurement -> Measurements;
        addMeasurement : Measurement -> Measurements -> Measurements;
        
        theball : Object;
        thetrain : Object;
        
        eKin : Measurable;
        mass : Measurable;
        
        meter : Unit;
        newton : Unit;
        gram : Unit;
        
        milli : Unit -> Unit;
        kilo : Unit -> Unit;
}

Defined Grammar

In [3]:
concrete GrammarEng of Grammar = {
  lincat
    Object = Str;
    Measurable = Str;
    Measurement = Str;
    Measurements = Str;
    Unit = Str;
    S = Str;
    
  lin
    state obj ms = obj ++ "has" ++ ms;
    unitCombine a b = a ++ b;
    measure m i u = "a" ++ m ++ "of" ++ i.s ++ u;
    toMeasurements m = m ;
    addMeasurement a b = a ++ "and" ++ b;
    theball = "the ball";
    thetrain = "the train";
    eKin = "kinetic energy";
    mass = "mass";
    meter = "m";
    newton = "N";
    gram = "g";
    milli u = "m" ++ u;
    kilo u = "k" ++ u;
}

In [4]:
generate_random -number=5 | linearize

the train has a kinetic energy of 999 g

the ball has a mass of 999 N N and a mass of 999 N

the ball has a mass of 999 N and a mass of 999 g

the train has a kinetic energy of 999 k N

the ball has a kinetic energy of 999 g g

In [5]:
parse "the ball has a mass of 5 k g"

state theball (toMeasurements (measure mass 5 (kilo gram)))

**The example sentence generates two parse trees**

In [6]:
parse "the ball has a mass of 5 k g and a kinetic energy of 12 m N"

state theball (addMeasurement (measure mass 5 (kilo gram)) (toMeasurements (measure eKin 12 (milli newton))))

state theball (addMeasurement (measure mass 5 (kilo gram)) (toMeasurements (measure eKin 12 (unitCombine meter newton))))

In [7]:
parse "the ball has a mass of 5 k g and a kinetic energy of 12 m N" | vt -nocat | show

2 graphs generated

Dropdown(description='Tree of:', layout=Layout(width='max-content'), options=('state theball (addMeasurement (…

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02)\x00\x00\x02\x1b\x08\x06\x00\x00\x00.!\x0e\xb8\x…

### Target Logic and Semantics Construction

In [8]:
theory LogicAndDT : ur:?LF =
    include ☞http://cds.omdoc.org/urtheories?NatSymbols ❙
    proposition : type ❘ # o ❙
    object : type ❘ # ι ❙
    unit : type ❘ # u ❙
    quantity : type ❘ # q ❙
    
    mult : u ⟶ u ⟶ u ❘ # 1 ⋅ 2 ❙
    quant : NAT ⟶ u ⟶ q ❙
    and : o ⟶ o ⟶ o ❘ # 1 ∧ 2 ❙
    
    ball : ι ❘ # theball ❙
    train : ι ❘ # thetrain ❙
    ekin : ι ⟶ q ⟶ o ❙
    mass : ι ⟶ q ⟶ o ❙
    newton : u ❘ # Newton ❙
    gram : u ❘ # gram ❙
    meter : u ❘ # meter ❙
    milli : u ⟶ u ❘ # milli 1 ❙
    kilo : u ⟶ u ❘ # kilo 1 ❙
❚

Created theory LogicAndDT

In [9]:
view SemConstr : http://mathhub.info/comma/jupyter/dim_analysis/Grammar.gf?Grammar -> ?LogicAndDT =
    NAT = NAT ❙
    zero = zero ❙

    Object = ι ❙
    Measurable = ι ⟶ q ⟶ o ❙
    Measurement = ι ⟶ o ❙
    Measurements = ι ⟶ o ❙
    Unit = u ❙
    S = o ❙

    // state : Object ⟶ Measurements ⟶ Sentence ❙
    state = [x, m] (m x) ❙
    // unitCombine : Unit ⟶ Unit ⟶ Unit ❙
    unitCombine = [a,b] a ⋅ b ❙
    // measure : Measurable ⟶ Int ⟶ Unit ⟶ Measurement ❙
    measure = [m,n,un] [x] m x (quant n un) ❙
    // simpleMeasurements : Measurement ⟶ Measurements ❙
    toMeasurements = [m] m ❙
    // addMeasurement : Measurement ⟶ Measurements ⟶ Measurements ❙
    addMeasurement = [m,ms] [x] (m x) ∧ (ms x) ❙
    // theball : Object ❙
    theball = ball ❙
    // thetrain : Object ❙
    thetrain = train ❙
    // eKin : Measurable ❙
    eKin = ekin ❙
    // mass : Measurable ❙
    mass = mass ❙
    // meter : Unit ❙
    meter = meter ❙
    // newton : Unit ❙
    newton = Newton ❙
    // gram : Unit ❙
    gram = gram ❙
    // milli : Unit ⟶ Unit ❙
    milli = milli ❙
    // kilo : Unit ⟶ Unit ❙
    kilo = kilo ❙
❚

Created view SemConstr

**We get two different logical expressions**

In [10]:
parse "the ball has a mass of 5 k g and a kinetic energy of 12 m N" | construct -v SemConstr

(mass theball (quant 5 kilo gram))∧(ekin theball (quant 12 milli Newton))

(mass theball (quant 5 kilo gram))∧(ekin theball (quant 12 meter⋅Newton))

### Dimensional Analysis with ELPI

In [11]:
elpi: dimAnalysis

% SIGNATURE
kind proposition type.
kind object type.
kind unit type.
kind quantity type.
type mult unit -> unit -> unit.
type udiv unit -> unit -> unit.
type quant int -> unit -> quantity.
type and proposition -> proposition -> proposition.
type ball object.
type train object.
type ekin object -> quantity -> proposition.
type mass object -> quantity -> proposition.
type newton unit.
type gram unit.
type meter unit.
type milli unit -> unit.
type kilo unit -> unit.


% DIMENSIONALITY STUFF

kind base_dimension type.
type length_dim base_dimension.
type mass_dim base_dimension.
type time_dim base_dimension.

kind dimension type.
% complex dimension, e.g. (cdim [length_dim] [time_dim, time_dim]) for speed
type cdim list base_dimension -> list base_dimension -> dimension.


% removes first occurence
type remove A -> list A -> list A -> prop.
remove X [X|L] L :- !.
remove X [H|L] [H|M] :- remove X L M.


type simplify_dim dimension -> dimension -> prop.
% TODO: std.mem! unnecessary
simplify_dim (cdim A B) R :- std.mem! A X, std.mem! B X, !, remove X A A2, remove X B B2, simplify_dim (cdim A2 B2) R.
simplify_dim (cdim A B) (cdim A B).


% convert unit to dimension
type utodim unit -> dimension -> prop.
utodim (mult A B) (cdim N1 N2) :- utodim A (cdim L1 L2), utodim B (cdim M1 M2), std.append L1 M1 N1, std.append L2 M2 N2.
utodim (udiv A B) (cdim N1 N2) :- utodim A (cdim L1 L2), utodim B (cdim M1 M2), std.append L1 M2 N1, std.append L2 M1 N2.
utodim (milli X) Y :- utodim X Y.
utodim (kilo X) Y :- utodim X Y.
utodim newton (cdim [mass_dim, length_dim] [time_dim, time_dim]).
% utodim joule (cdim [length_dim, mass_dim, length_dim] [time_dim, time_dim]).
utodim gram (cdim [mass_dim] []).
utodim meter (cdim [length_dim] []).

type dim_eq dimension -> dimension -> prop.
dim_eq (cdim [] []) (cdim [] []).
% TODO: std.mem! unnecessary
dim_eq (cdim L LL) (cdim M MM) :- std.mem! L X, std.mem! M X, remove X L L2, remove X M M2, dim_eq (cdim L2 LL) (cdim M2 MM).
dim_eq (cdim [] L) (cdim [] M) :- std.mem! L X, std.mem! M X, remove X L L2, remove X M M2, dim_eq (cdim [] L2) (cdim [] M2).


% check that quantity has specific dimension.
type dimcheck quantity -> dimension -> prop.
dimcheck (quant _ X) D :-
    utodim X E, 
    simplify_dim D D2,
    simplify_dim E E2,
    dim_eq D2 E2, !.

dimcheck (quant _ X) D :-
    utodim X E, 
    simplify_dim E E2,
    simplify_dim D D2,
    dimstr D2 DS,
    dimstr E2 ES,
    print "REJECTED:" X "has dimension" ES "but expected" DS,
    fail.


% PRINTING

% stringjoin ", " ["a", "b", "c"] "a, b, c"
type stringjoin string -> list string -> string -> prop.
stringjoin _ [] "".
stringjoin _ [S] S :- !.
stringjoin Sep [H|T] S :- stringjoin Sep T S2, S is H ^ Sep ^ S2.

type basedimstr base_dimension -> string -> prop.
basedimstr length_dim "length".
basedimstr mass_dim "mass".
basedimstr time_dim "time".

type halfdimstr list base_dimension -> string -> prop.
halfdimstr X R :- std.map X (x \ basedimstr x) XS, stringjoin "*" XS R.

type dimstr dimension -> string -> prop.
dimstr (cdim [A1|A] [B1,B2|B]) S :- !, halfdimstr [A1|A] AS, halfdimstr [B1,B2|B] BS, S is AS ^ "/(" ^ BS ^ ")".
dimstr (cdim [A1|A] [B1|B]) S :- !, halfdimstr [A1|A] AS, halfdimstr [B1|B] BS, S is AS ^ "/" ^ BS.
dimstr (cdim [A1|A] []) S :- !, halfdimstr [A1|A] R, S is R.
dimstr (cdim [] [B1|B]) S :- !, halfdimstr [B1|B] R, S is  "1" ^ "/" ^ R.
dimstr (cdim [] []) S :- S is "1".

type checkactual proposition -> prop.
checkactual (and A B) :- checkactual A, checkactual B.
checkactual (ekin _ Q) :- dimcheck Q (cdim [length_dim, length_dim, mass_dim] [time_dim, time_dim]).
checkactual (mass _ Q) :- dimcheck Q (cdim [mass_dim] []).


type check list string -> prop.
check [X] :- print X, string_to_term X XT, checkactual XT, print "ACCEPTED".

Created dimAnalysis.elpi

**The wrong reading is rejected**

In [12]:
parse "the ball has a kinetic energy of 12 m N"

state theball (toMeasurements (measure eKin 12 (milli newton)))

state theball (toMeasurements (measure eKin 12 (unitCombine meter newton)))

In [13]:
parse "the ball has a kinetic energy of 12 m N" | construct -elpi

(ekin ball (quant 12 (milli newton)))

(ekin ball (quant 12 (mult meter newton)))

In [14]:
parse "the ball has a kinetic energy of 12 m N" | construct -elpi | elpi dimAnalysis check

(ekin ball (quant 12 (milli newton)))
REJECTED: milli newton has dimension mass*length/(time*time) but expected 
 length*length*mass/(time*time)


(ekin ball (quant 12 (mult meter newton)))
ACCEPTED


In [15]:
parse -cat=S "the ball has a mass of 5 k g and a kinetic energy of 12 m N" | construct -elpi | elpi dimAnalysis check

(and (mass ball (quant 5 (kilo gram))) (ekin ball (quant 12 (milli newton))))
REJECTED: milli newton has dimension mass*length/(time*time) but expected 
 length*length*mass/(time*time)


(and (mass ball (quant 5 (kilo gram))) (ekin ball (quant 12 (mult meter newton))))
ACCEPTED
